In [237]:
from scipy.stats import gumbel_r

from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation, performance_metrics
from fbprophet.plot import plot_cross_validation_metric

from sklearn.metrics import mean_absolute_error
import numpy as np


import os
import pandas as pd
from utils import (
    ESTACIONES_XLSX,
    NCEP_NCAR_XLSX,
    NDEFM_XLSX,
    PREDICTIONS_FOLDER,
    download_file,
    DATA_FOLDER
)

In [125]:
df_maximos = pd.read_excel(ESTACIONES_XLSX, sheet_name=1, header=1, parse_dates=['Año hid'])
df_maximos = df_maximos[['Año hid', 'San Luis Tucuman']]
df_maximos = df_maximos[((~df_maximos['San Luis Tucuman'].isnull()).cumsum() == 0).idxmin():]
df_maximos = df_maximos[['Año hid', 'San Luis Tucuman']].rename(columns={'Año hid':'ds', 'San Luis Tucuman':'y'})

In [234]:
eps = 1e-4
returns = [round(1-1/x, 2)+eps for x in range(1, 11)]

def gumbel_r_mom(x):
    scale = np.sqrt(6)/np.pi * np.std(x)
    loc = np.mean(x) - np.euler_gamma*scale
    return loc, scale

loc, scale = gumbel_r_mom(df_maximos.y)

gumb = gumbel_r(loc=loc, scale=scale)

In [225]:
pd.DataFrame(gumb.ppf(returns)).to_csv(os.path.join(PREDICTIONS_FOLDER, 'gumbel_ppf.csv'), index=True, header=False)

In [227]:
pd.DataFrame(gumb.rvs(size=10)).to_csv(os.path.join(PREDICTIONS_FOLDER, 'gumbel_rvs.csv'), index=True, header=False)

# Con la data de Luciano

In [238]:
download_file("https://raw.githubusercontent.com/LucianoBrum/Metadata_lluvias/master/san_luis_entero.csv",
             os.path.join(DATA_FOLDER, 'san_luis_entero.csv'),
)

In [244]:
df_maximos = pd.read_csv(os.path.join(DATA_FOLDER, 'san_luis_entero.csv'), index_col=0).rename(columns={'Año hid':'ds', 'San Luis Tucuman':'y'})

In [246]:
eps = 1e-4
returns = [round(1-1/x, 2)+eps for x in range(1, 11)]

def gumbel_r_mom(x):
    scale = np.sqrt(6)/np.pi * np.std(x)
    loc = np.mean(x) - np.euler_gamma*scale
    return loc, scale

loc, scale = gumbel_r_mom(df_maximos.y)

gumb = gumbel_r(loc=loc, scale=scale)

In [249]:
pd.DataFrame(gumb.ppf(returns)).to_csv(os.path.join(PREDICTIONS_FOLDER, 'gumbel_ppf_luc.csv'), index=True, header=False)

In [250]:
pd.DataFrame(gumb.rvs(size=10)).to_csv(os.path.join(PREDICTIONS_FOLDER, 'gumbel_rvs_luc.csv'), index=True, header=False)